In [3]:
from code_app.obtener_precedentes import obtener_precedentes_completos
from code_app.filtrar_raices import filtrar_valores_raiz
from code_app.simular_impacto_raices import simular_impacto_raices
from code_app.filtrar_impactos import filtrar_mayores_impactos
from tabulate import tabulate  # pip install tabulate

In [4]:
ruta_excel = "/Users/alvarofelipepupuchemorales/Desktop/Proyecto Economia/assets/PALTA HASS  BCP.xlsx"
hoja_nombre = "Costos Agricolas 01 ha"
celda_objetivo = "E123"
# Conseguimos los precedentes
precedentes_totales = obtener_precedentes_completos(ruta_excel, hoja_nombre, celda_objetivo)
for p in precedentes_totales:
    print(p)

Costos Agricolas 01 ha!$E$45
Costos Agricolas 01 ha!$E$46
Costos Agricolas 01 ha!$E$48
Costos Agricolas 01 ha!$E$53
Costos Agricolas 01 ha!$E$55
Costos Agricolas 01 ha!$H$95
Costos Agricolas 01 ha!E11
Costos Agricolas 01 ha!E120
Costos Agricolas 01 ha!E29
Costos Agricolas 01 ha!E30
Costos Agricolas 01 ha!E31
Costos Agricolas 01 ha!E32
Costos Agricolas 01 ha!E33
Costos Agricolas 01 ha!E34
Costos Agricolas 01 ha!E41
Costos Agricolas 01 ha!E46
Costos Agricolas 01 ha!E50
Costos Agricolas 01 ha!E53
Costos Agricolas 01 ha!E84
Costos Agricolas 01 ha!E85
Costos Agricolas 01 ha!E86
Costos Agricolas 01 ha!E87
Costos Agricolas 01 ha!E88
Costos Agricolas 01 ha!E89
Costos Agricolas 01 ha!F11
Costos Agricolas 01 ha!F120
Costos Agricolas 01 ha!F15
Costos Agricolas 01 ha!F22
Costos Agricolas 01 ha!F23
Costos Agricolas 01 ha!F31
Costos Agricolas 01 ha!F41
Costos Agricolas 01 ha!F84
Costos Agricolas 01 ha!F85
Costos Agricolas 01 ha!F86
Costos Agricolas 01 ha!F87
Costos Agricolas 01 ha!F88
Costos Agricol

In [82]:
print("="*100)

raices = filtrar_valores_raiz(ruta_excel, precedentes_totales)

print("Raíces encontradas:")
print(len(raices))
for celda, valor in raices.items():
    print(f"{celda} → {valor}")

Raíces encontradas:
50
Costos Agricolas 01 ha!$E$45 → 25
Costos Agricolas 01 ha!$E$46 → 0.9
Costos Agricolas 01 ha!$E$53 → 1600
Costos Agricolas 01 ha!$H$95 → 0.07
Costos Agricolas 01 ha!E29 → 100
Costos Agricolas 01 ha!E46 → 0.9
Costos Agricolas 01 ha!E50 → 0.02
Costos Agricolas 01 ha!E53 → 1600
Costos Agricolas 01 ha!F98 → 30800
Costos Agricolas 01 ha!G15 → 350
Costos Agricolas 01 ha!G16 → 200
Costos Agricolas 01 ha!G17 → 1000
Costos Agricolas 01 ha!G19 → 121
Costos Agricolas 01 ha!G23 → 1200
Costos Agricolas 01 ha!G24 → 750
Costos Agricolas 01 ha!G25 → 200
Costos Agricolas 01 ha!G37 → 200
Costos Agricolas 01 ha!G57 → 0
Costos Agricolas 01 ha!G85 → 0
Costos Agricolas 01 ha!G88 → 3800
Costos Agricolas 01 ha!H95 → 0.07
Costos Agricolas 01 ha!J94 → 30800
Costos Agricolas 01 ha!V15 → 427
Costos Agricolas 01 ha!V17 → 1734
Costos Agricolas 01 ha!V19 → 480
Costos Agricolas 01 ha!V23 → 2200
Costos Agricolas 01 ha!V24 → 1200
Costos Agricolas 01 ha!V37 → 300
INSTALACION Y SIEMBRA!B19 → 900
INS

In [34]:
import openpyxl

In [221]:
import openpyxl
import re

def generar_precedentes(ruta_excel, hoja_objetivo, celda_objetivo):
    wb = openpyxl.load_workbook(ruta_excel, data_only=False)
    hoja = wb[hoja_objetivo]

    mapa = {}
    formulas = {}
    raices = {}
    visitadas = set()

    def npv(rate, values):
        return sum(v / (1 + rate) ** (i + 1) for i, v in enumerate(values))

    def obtener_formula(celda):
        valor = hoja[celda].value
        if hoja[celda].data_type == 'f':
            return valor
        return None

    def limpiar_formula(formula):
        if formula.startswith('='):
            formula = formula[1:]
        if formula.startswith('+'):
            formula = formula[1:]
        return formula

    def extraer_referencias(formula):
        sin_hoja = re.sub(r"(?:'[^']+'|[A-Za-z0-9_]+)!", "", formula)
        return re.findall(r'\b[A-Z]{1,3}[0-9]{1,7}\b', sin_hoja)

    def expandir_rango(rango):
        from openpyxl.utils import column_index_from_string, get_column_letter
        start, end = rango.split(":")
        start_col, start_row = re.match(r'([A-Z]+)([0-9]+)', start).groups()
        end_col, end_row = re.match(r'([A-Z]+)([0-9]+)', end).groups()

        cols = list(range(column_index_from_string(start_col), column_index_from_string(end_col) + 1))
        rows = list(range(int(start_row), int(end_row) + 1))

        resultado = [f"{get_column_letter(col)}{row}" for col in cols for row in rows]
        return resultado

    def procesar_celda(celda):
        if celda in visitadas:
            return
        visitadas.add(celda)

        formula = obtener_formula(celda)
        if not formula:
            raices[celda] = hoja[celda].value
            return

        formula_limpia = limpiar_formula(formula)
        formula_sin_hoja = re.sub(r"(?:'[^']+'|[A-Za-z0-9_]+)!", "", formula_limpia)

        # Expandir rangos y usar placeholders temporales
        rangos = re.findall(r'\$?[A-Z]{1,3}\$?[0-9]+:\$?[A-Z]{1,3}\$?[0-9]+', formula_sin_hoja)
        placeholders = {}
        for i, rango in enumerate(rangos):
            celdas = expandir_rango(rango.replace('$', ''))
            reemplazo = "[" + ",".join(f"r('{c}')" for c in celdas) + "]"
            placeholder = f"__RANGO_{i}__"
            formula_sin_hoja = formula_sin_hoja.replace(rango, placeholder)
            placeholders[placeholder] = reemplazo

        referencias = extraer_referencias(formula_sin_hoja)
        referencias = [ref.replace('$', '') for ref in referencias]

        for ref in referencias:
            procesar_celda(ref)

        # Reemplazar referencias simples
        for ref in referencias:
            formula_sin_hoja = re.sub(rf'\b{ref}\b', f"r('{ref}')", formula_sin_hoja)

        # Restaurar los rangos
        for placeholder, reemplazo in placeholders.items():
            formula_sin_hoja = formula_sin_hoja.replace(placeholder, reemplazo)

        formulas[celda] = formula_sin_hoja

        try:
            mapa[celda] = eval(f"lambda r: {formula_sin_hoja}", {
                "npv": npv,
                "sum": sum,
                "max": max,
                "min": min,
                "abs": abs,
                "NPV" : npv
            })
        except Exception as e:
            print(f"[ERROR] al evaluar fórmula en {celda}: {formula_sin_hoja}")
            print(f"       {e}")

    procesar_celda(celda_objetivo)
    return mapa, formulas, raices

In [222]:
mapa, formulas, raices = generar_precedentes(ruta_excel, hoja_nombre, celda_objetivo)

print(mapa)
print(formulas)
print(raices)

{'H26': <function <lambda> at 0x115997f40>, 'E30': <function <lambda> at 0x115994ca0>, 'E31': <function <lambda> at 0x1159968c0>, 'H11': <function <lambda> at 0x115994ee0>, 'H38': <function <lambda> at 0x115996050>, 'H39': <function <lambda> at 0x115997760>, 'H36': <function <lambda> at 0x115996a70>, 'E32': <function <lambda> at 0x115db27a0>, 'E33': <function <lambda> at 0x115db2b90>, 'E34': <function <lambda> at 0x115db2200>, 'E11': <function <lambda> at 0x115db0b80>, 'E41': <function <lambda> at 0x115db15a0>, 'E87': <function <lambda> at 0x115db2560>, 'E88': <function <lambda> at 0x115db3760>, 'E89': <function <lambda> at 0x115db32e0>, 'E120': <function <lambda> at 0x115db1fc0>, 'F15': <function <lambda> at 0x115db2dd0>, 'F22': <function <lambda> at 0x115db2a70>, 'F23': <function <lambda> at 0x115db2320>, 'F31': <function <lambda> at 0x115db1ea0>, 'F11': <function <lambda> at 0x115db1b40>, 'F41': <function <lambda> at 0x11545de10>, 'F87': <function <lambda> at 0x11545c700>, 'F89': <f

In [223]:
print(celda_objetivo in mapa)

True


In [224]:
def simular_impacto_raices_sin_excel(celda_objetivo, mapa, raices, formulas):
    resultados = {}

    def evaluar(celda, r, mapa, cache={}):
        if celda in cache:
            return cache[celda]
        if celda in r:
            cache[celda] = r[celda]
            return r[celda]
        elif celda in mapa:
            valor = mapa[celda](lambda c: evaluar(c, r, mapa, cache))
            cache[celda] = valor
            return valor
        else:
            raise ValueError(f"No se puede evaluar la celda: {celda}")

    # Evaluar valor base (sin variaciones)
    valor_base = evaluar(celda_objetivo, raices, mapa)

    # Simular impacto de variar cada raíz en +1%
    for raiz in raices:
        copia_raices = raices.copy()
        copia_raices[raiz] *= 1.01  # +1%

        try:
            valor_nuevo = evaluar(celda_objetivo, copia_raices, mapa)
            diferencia_abs = valor_nuevo - valor_base
            diferencia_rel = (diferencia_abs / valor_base) * 100 if valor_base != 0 else 0

            resultados[raiz] = {
                "valor_base": valor_base,
                "valor_nuevo": valor_nuevo,
                "variacion_absoluta": diferencia_abs,
                "variacion_porcentual": diferencia_rel,
            }

        except Exception as e:
            resultados[raiz] = {
                "error": str(e)
            }

    return resultados

In [225]:


# Obtener resultados
impacto = simular_impacto_raices_sin_excel("E123", mapa, raices, formulas)

print(impacto)

ValueError: No se puede evaluar la celda: G120